In [ ]:
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import pickle
import os
import pandas as pd
from tqdm import tqdm
from scipy.optimize import curve_fit
#缺失数据做不到

In [ ]:
with open('/data6/peiyan/SH-METR/results/userLabels.pkl','rb') as infile: # {1 : commuter, 0 : non-commuter, -1 : no home}
    userLabels = pickle.load(infile)

with open('/data6/peiyan/SH-METR/results/stationLabels.pkl','rb') as infile: # h,w,0
    tationLabels = pickle.load(infile)

with open('/data6/peiyan/SH-METR/results/TripLabels.pkl','rb') as infile:  # 0: c, 1: HBO, 2+3: NHB
    tripLabels = pickle.load(infile)

In [2]:
stationInfo = pd.read_csv('../../Metro/station_transInfo_cleaned.csv')
dict_staionName = dict(zip(stationInfo['stationID'], stationInfo['name']))
dict_stationLoc = dict(zip(stationInfo['stationID'], zip(stationInfo['lon'],stationInfo['lat'])))
stationID_List = list(stationInfo['stationID'])
stationID2index = {ID:index for index, ID in enumerate(stationID_List)}

with open('../../MetaData/travelDis.pkl', 'rb') as f:
    travelDis = pickle.load(f)

with open('../../MetaData/travelTime.pkl','rb') as f:
    travelTime = pickle.load(f)

In [4]:
numCommuters = np.sum([i for i in userLabels.values() if i==1])
numNoHome = np.sum([-i for i in userLabels.values() if i==-1])
numNonCommuters = len(userLabels) - numCommuters - numNoHome

print( "# of commuters : {}".format(numCommuters/float(numNonCommuters + numCommuters)))
print("# of non-commuters : {}".format(numNonCommuters/float(numNonCommuters + numCommuters)))
print ("# of no home : {}".format(numNoHome/float(len(userLabels))))


# of commuters : 0.6727500203838562
# of non-commuters : 0.32724997961614377
# of no home : 0.25710860000060926


In [7]:
# distribution of trip distances & time for all, commuters, non-commuters
# distances = pickle.load(open("results/stationDistances.pkl", 'rb'))
allTripsDistances = [] 
allTripsTimes = []      
distancesInTripTypes = [[] for i in range(3)] # 0:c, 1: HBO, 2:NHB
timesInTripTypes = [[] for i in range(3)]

# describe activeness of user and station by trip numbers 
userTripCount = []
stationTripCount = [ 0 for i in range(len(stationID_List))]

for user in tqdm(userLabels):
    userTripCount.append(len(tripLabels[user]))
    label = userLabels[user]
    for trip, tripL in tripLabels[user].items(): #trip:(inStation, outStation), tripL: tripLabel 
        inStationID = trip[0]
        outStationID = trip[1]
        instationIndex = stationID2index[inStationID]
        outstationIndex = stationID2index[outStationID]
        #dist = distances[trip]
        dist = travelDis[instationIndex, outstationIndex]/1000 # m -> km
        time = travelTime[instationIndex, outstationIndex]/60 # second -> minute
        allTripsDistances.append(dist)
        allTripsTimes.append(time)

        tripL_v2 = tripL if tripL<2 else 2
        distancesInTripTypes[tripL_v2].append(dist)
        timesInTripTypes[tripL_v2].append(time)

        stationTripCount[instationIndex] += 1
        stationTripCount[outstationIndex] += 1

print(np.max(allTripsDistances))
print(np.max(allTripsTimes))

with open('temp/statistics_trip.pkl','wb') as outfile:
    pickle.dump([allTripsDistances, distancesInTripTypes, allTripsTimes, timesInTripTypes, userTripCount, stationTripCount], outfile)

  0%|          | 0/19695580 [00:00<?, ?it/s]

100%|██████████| 19695580/19695580 [08:33<00:00, 38382.79it/s]


115.265
264.87916666666666


In [3]:
with open('temp/statistics_trip.pkl','rb') as infile:
    allTripsDistances, distancesInTripTypes, allTripsTimes, timesInTripTypes, userTripCount, stationTripCount = pickle.load(infile)

In [100]:
# plot the distritbuion
# travel distance & time of varying types of trips

# travel distance of varying types of trips

def truncated_power_law(x, C, alpha, lambd):
    return C  * (x ** (-alpha)) *np.exp(-lambd * x)

maxV = 120
numInterval = 40
# bins = np.linspace(0, maxV, numInterval + 1)
log_bins = np.logspace(np.log10(1), np.log10(maxV), numInterval + 1)
bin_centers = np.sqrt(log_bins[:-1] * log_bins[1:]) 

usagesHist = np.histogram(np.array(allTripsDistances),log_bins)
usagesHist_all = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))

# home-work trip
usagesHist = np.histogram(np.array(distancesInTripTypes[0]), log_bins)
usagesHist_HW = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))

# home-other trip
usagesHist = np.histogram(np.array(distancesInTripTypes[1]), log_bins)
usagesHist_HO = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))

# none-home-based trip
usagesHist = np.histogram(np.array(distancesInTripTypes[2]), log_bins)
usagesHist_NHB = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))

bins = np.array(bins[1:])

fig = plt.figure(figsize=(6, 4))
ax = plt.subplot(1, 1, 1)

plt.scatter(bin_centers.tolist(), usagesHist_all.tolist(), marker='o',s=15, linewidth=0.5, facecolor='#41A7D8',
        edgecolor='k', label='All')
plt.scatter(bin_centers.tolist(), usagesHist_HW.tolist(), marker='v', s=15, linewidth=0.5, facecolor='#1b7837',
        edgecolor='k', label='Commuting Trips')
plt.scatter(bin_centers.tolist(), usagesHist_HO.tolist(), marker='s', s=15, linewidth=0.5, facecolor='#d73027',
        edgecolor='k', label='Home Based Other Trips')
plt.scatter(bin_centers.tolist(), usagesHist_NHB.tolist(), marker='s', s=15, linewidth=0.5, facecolor='#762a83',
        edgecolor='k', label='None Home Based Trips')

plt.xlim(1, maxV)
plt.ylim(1e-5)
#plt.xticks(np.linspace(0, maxV, numInterval/10+1))
#plt.xticks(np.linspace(0, int(maxV), int(numInterval/10+1)))
plt.xticks(bin_centers)
plt.xscale("log", nonpositive='clip')
plt.yscale("log", nonpositive='clip')
plt.xlabel(r'Travel distance, d [km]', fontsize=12)
plt.ylabel(r"Density, $F(d)$", fontsize=12)
plt.tick_params(labelsize=12)


# fit curve
x_data = bin_centers[27:]
y_data = usagesHist_all[27:]

p0 = [1.0, 0.1, 0.01]  # Initial guess for parameters [C, alpha, λ]
bounds = ([0, 0, 0], [np.inf, np.inf, np.inf])  # Parameter bounds
params, _ = curve_fit(truncated_power_law, x_data, y_data, p0=p0, bounds=bounds)
print(params)

# Generate points for the fitted curve
x_fit = np.logspace(np.log10(bin_centers[19]), np.log10(maxV), 100)  
y_fit = truncated_power_law(x_fit, *params)
C = params[0].round(2)
alpha = params[1].round(18)
lambd = params[2].round(4)
plt.plot(x_fit, y_fit, '--', color='#00856D', linewidth=2, label=fr'$p(d) = {C} e^{{-{lambd}d}}$', zorder=0)
plt.legend(loc="lower left", frameon=False)


plt.tight_layout()
#plt.savefig('../results/distance_tripType.png', dpi=300)
plt.savefig('../results/distance_tripType.pdf')
plt.close()



[1.79084563e-01 5.55106615e-17 5.68487296e-02]


In [4]:
import seaborn as sns
# Activeness of users and stations
fig, ax = plt.subplots(1,2, figsize=(8, 4))
print(np.max(userTripCount))
print(np.max(stationTripCount))

sns.histplot(userTripCount, bins=np.arange(0, 120, 4), kde=True, color='royalblue', edgecolor='white',linewidth=1.2, ax=ax[0])
ax[0].set_xlabel('# of trips by users',fontsize=12)
ax[0].set_ylabel('User count',fontsize=12)
ax[0].set_yscale('log')
ax[0].set_xlim(-5, 120)
ax[0].set_ylim(1e4, 1.5e7)
ax[0].tick_params(labelsize=12)
#ax[0].spines['top'].set_visible(False)
#ax[0].spines['right'].set_visible(False)

sns.histplot(stationTripCount, bins=np.arange(0, 1e7, 500000), kde=True, color='salmon', edgecolor='white', linewidth=1.2,ax=ax[1])
ax[1].set_xlabel('# of trips by stations',fontsize=12)
ax[1].set_ylabel('Station count',fontsize=12)
ax[1].tick_params(labelsize=12)
#ax[1].spines['top'].set_visible(False)
#ax[1].spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig('../results/activeness_analysis.pdf', dpi=300)
plt.close()


661
13409060
